In [ ]:
import os
import pandas as pd
import numpy as np
import datetime
import pickle
pd.set_option('display.max_columns', None)
pd.set_option("display.max_rows", 50)

import warnings

# Use this filter to ignore the specific FutureWarning
warnings.simplefilter(action='ignore', category=FutureWarning)

target_folder_path = '/Volumes/Database/Original_data'
%cd "{target_folder_path}"

/Volumes/Database/Original_data


### Get correct_datetime_df "open"

In [1]:
open = pd.read_csv("/Volumes/Database/Factor_data/open.csv", index_col=0)

NameError: name 'pd' is not defined

In [3]:
rev_relative_data = pd.read_table(rf"{target_folder_path}/Company_DB/月營收盈餘.txt", encoding='Big5-HKSCS', low_memory=False)

In [4]:
columns_list = rev_relative_data.columns.tolist()
the_place_start_to_become_numeric = 4

### make all_columns to str
for column in columns_list:
    rev_relative_data[column] = rev_relative_data[column].astype(str).str.strip()

### make numeric data to be numeric
numeric_list = [element for element in columns_list[the_place_start_to_become_numeric:] if "發布日" not in element] # [2:] is become I need to make the future symbol columns "ticker" keep in str
for elements in numeric_list:
    rev_relative_data[elements] = pd.to_numeric(rev_relative_data[elements], errors='coerce')

### make date inf to datetime
rev_relative_data['年月'] = pd.to_datetime(rev_relative_data['年月'], format='%Y%m')
elements_with_releseday = [element for element in columns_list if "發布日" in element]
for elements in elements_with_releseday:
    rev_relative_data[elements] = pd.to_datetime(rev_relative_data[elements], format='%Y%m%d')

rev_relative_data.rename(columns= {'公司': 'symbol',
                                   '年月': 'datetime',
                                   '營收發布日': 'release'}, inplace=True)

In [5]:
rev_relative_data.columns = rev_relative_data.columns.str.replace('/', '')
output_list = [col for col in rev_relative_data.columns[the_place_start_to_become_numeric:] if col not in elements_with_releseday]

In [6]:
# # 100% according to publish date (date rebalance)
# for column_name in output_list:
#     print(column_name)
#     unstackdata_release = rev_relative_data.set_index(['datetime','symbol'])['release'].unstack().loc['2013':]
#     unstackdata_imf = rev_relative_data.set_index(['datetime','symbol'])[column_name].unstack().loc['2013':]
    
#     unstackdata_release = unstackdata_release.reindex(open.index).fillna(method='ffill')
#     filter = unstackdata_release.apply(lambda x: x == x.index)
#     unstackdata_imf = unstackdata_imf.reindex(open.index).fillna(method='ffill')
#     unstackdata_imf_filtered = unstackdata_imf[filter]
#     data_reindex = unstackdata_imf_filtered.fillna(method='ffill', inplace=False)
#     csv_file = os.path.join('/Volumes/Database/Factor_data', f'{column_name}_100.csv')
#     data_reindex.to_csv(csv_file)

In [45]:
### 不再更新的資料會持續向下ffill產生error --> 會持續到隔年初被判定
# 90% first release (quarter rebalance)
unstackdata_release = rev_relative_data.reset_index().set_index(['datetime','symbol'])['release'].unstack().loc['2013':]
open_indexes = open.loc['2013':].index

for column_name in output_list:
    unstackdata_imf = rev_relative_data.reset_index().set_index(['datetime','symbol'])[column_name].unstack().loc['2013':]
    releaserankpct = unstackdata_release.rank(axis=1,pct=True)
    filter = unstackdata_imf[releaserankpct < 0.9]
    filter.index = unstackdata_release[releaserankpct < 0.9].max(axis=1)
    data_reindex = filter.reindex(open_indexes, method = 'ffill') # 不同於fillna(ffill) 對NaN的處理不同
    csv_file = os.path.join('/Volumes/Database/Factor_data', f'{column_name}_90.csv')
    data_reindex.to_csv(csv_file)

In [ ]:
# # 80% first release (quarter rebalance)
# for column_name in output_list:
#     unstackdata_release = rev_relative_data.set_index(['datetime','symbol'])['release'].unstack().loc['2013':]
#     unstackdata_imf = rev_relative_data.set_index(['datetime','symbol'])[column_name].unstack().loc['2013':]
#     unstackdata_release = unstackdata_release.reindex(open.index).fillna(method='ffill')
#     unstackdata_imf = unstackdata_imf.reindex(open.index).fillna(method='ffill')
#     publushrankpct = unstackdata_release.rank(axis=1,pct=True)
#     data_reindex = unstackdata_imf[publushrankpct < 0.8]
#     csv_file = os.path.join('/Volumes/Database/Factor_data', f'{column_name}_90.csv')
#     data_reindex.to_csv(csv_file)

### check

In [ ]:
open = pd.read_csv("/Volumes/Database/Factor_data/單月營收(千元)_90.csv", index_col=0)
open

000116    000930     000960     0009A0       1101       1102  \
datetime                                                                     
1980-01-04       NaN       NaN        NaN        NaN        NaN        NaN   
1980-01-05       NaN       NaN        NaN        NaN        NaN        NaN   
1980-01-07       NaN       NaN        NaN        NaN        NaN        NaN   
1980-01-08       NaN       NaN        NaN        NaN        NaN        NaN   
1980-01-09       NaN       NaN        NaN        NaN        NaN        NaN   
...              ...       ...        ...        ...        ...        ...   
2024-03-06  461716.0  466635.0  1952634.0  1482048.0  9514039.0  6313936.0   
2024-03-07  461716.0  466635.0  1952634.0  1265646.0  9514039.0  6313936.0   
2024-03-08  461716.0  466635.0  1602660.0  1265646.0  4798581.0  3599986.0   
2024-03-11  461716.0  466635.0  1602660.0  1265646.0  4798581.0  3599986.0   
2024-03-12  461716.0  466635.0  1602660.0  1265646.0  4798581.0  3599986.0   

                1103      1104   1107      1108      1109      1110  1113  \
datetime                                                                    
1980-01-04       NaN       NaN    NaN       NaN       NaN       NaN   NaN   
1980-01-05       NaN       NaN    NaN       NaN       NaN       NaN   NaN   
1980-01-07       NaN       NaN    NaN       NaN       NaN       NaN   NaN   
1980-01-08       NaN       NaN    NaN       NaN       NaN       NaN   NaN   
1980-01-09       NaN       NaN    NaN       NaN       NaN       NaN   NaN   
...              ...       ...    ...       ...       ...       ...   ...   
2024-03-06  283279.0  742480.0  216.0  449490.0  569161.0  211257.0   NaN   
2024-03-07  283279.0  742480.0  216.0  286337.0  569161.0  211257.0   NaN   
2024-03-08  283279.0  742480.0  216.0  286337.0  226768.0  211257.0   NaN   
2024-03-11  201295.0  477355.0  216.0  286337.0  226768.0  175185.0   NaN   
2024-03-12  201295.0  477355.0  216.0  286337.0  226768.0  175185.0   NaN   

                 1201      1203  1204  1206  1207  1208  1209       1210  \
datetime                                                                   
1980-01-04        NaN       NaN   NaN   NaN   NaN   NaN   NaN        NaN   
1980-01-05        NaN       NaN   NaN   NaN   NaN   NaN   NaN        NaN   
1980-01-07        NaN       NaN   NaN   NaN   NaN   NaN   NaN        NaN   
1980-01-08        NaN       NaN   NaN   NaN   NaN   NaN   NaN        NaN   
1980-01-09        NaN       NaN   NaN   NaN   NaN   NaN   NaN        NaN   
...               ...       ...   ...   ...   ...   ...   ...        ...   
2024-03-06  1736336.0  559442.0   NaN   NaN   NaN   NaN   NaN  9270160.0   
2024-03-07  1736336.0  559442.0   NaN   NaN   NaN   NaN   NaN  9270160.0   
2024-03-08  1736336.0  559442.0   NaN   NaN   NaN   NaN   NaN  9270160.0   
2024-03-11  1730379.0  472834.0   NaN   NaN   NaN   NaN   NaN  7242006.0   
2024-03-12  1730379.0  472834.0   NaN   NaN   NaN   NaN   NaN  7242006.0   

            1212     1213       1215        1216      1217      1218  \
datetime                                                               
1980-01-04   NaN      NaN        NaN         NaN       NaN       NaN   
1980-01-05   NaN      NaN        NaN         NaN       NaN       NaN   
1980-01-07   NaN      NaN        NaN         NaN       NaN       NaN   
1980-01-08   NaN      NaN        NaN         NaN       NaN       NaN   
1980-01-09   NaN      NaN        NaN         NaN       NaN       NaN   
...          ...      ...        ...         ...       ...       ...   
2024-03-06   NaN   3523.0  2407927.0  49174331.0  311678.0  753291.0   
2024-03-07   NaN   3523.0  2407927.0  49174331.0  311678.0  753291.0   
2024-03-08   NaN  14504.0  2030911.0  49174331.0  311678.0  787943.0   
2024-03-11   NaN  14504.0  2030911.0  47898965.0  323591.0  787943.0   
2024-03-12   NaN  14504.0  2030911.0  47898965.0  323591.0  787943.0   

                 1219      1220  1221  1222  1224       1225       1227